In [1]:
from bs4 import BeautifulSoup
from collections import Counter, defaultdict
import json
from tqdm import tqdm

In [2]:
with open("data/href2dramapage.json", encoding="utf-8") as f:
    href2dramapage = json.load(f)

In [15]:
href2details = defaultdict(dict)

for href, page in tqdm(href2dramapage.items()):
    if href not in href2details:
        
        soup = BeautifulSoup(page)
        details = soup.find_all("ul", {"class": "list m-b-0"})
        if details:
            
            for details_block in details:
                separate_details = details_block.find_all("li", {"class": "list-item p-a-0"})
                
                for separate_detail in separate_details:
                    pass
                    
        
    break

  0%|                                                                                        | 0/19462 [00:00<?, ?it/s]


In [16]:
separate_details[0]

<li class="list-item p-a-0"><b class="inline">Score:</b> 8.2 <span class="hft">(scored by 12 users)</span></li>

In [17]:
details[1]

<ul class="list m-b-0"> <li class="list-item p-a-0"><b class="inline">Score:</b> 8.2 <span class="hft">(scored by 12 users)</span></li> <li class="list-item p-a-0"><b class="inline">Ranked:</b> #37553</li> <li class="list-item p-a-0"><b class="inline">Popularity:</b> #12666</li> <li class="list-item p-a-0"><b class="inline">Watchers:</b> 302</li> <li class="list-item p-a-0" style="display:none;"><b class="inline">Favorites:</b> 0</li> </ul>